In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
import pathlib
import json
import numpy as np
from ophys_etl.modules.decrosstalk.ophys_plane import OphysROI, OphysMovie
from ophys_etl.modules.decrosstalk.ophys_plane import find_overlapping_roi_pairs
from ophys_etl.modules.segmentation.qc_utils.roi_utils import ROIExaminer
from ophys_etl.modules.segmentation.qc_utils.roi_utils import add_roi_boundaries_to_img
from ophys_etl.modules.segmentation.qc_utils.roi_utils import roi_thumbnail

In [ ]:
data_dir = pathlib.Path('/Users/scott.daniel/Pika/deep_interpolation/data')
assert data_dir.is_dir()

In [ ]:
movie_path = data_dir / 'denoised.h5'
roi_path = data_dir / 'denoised_segmentation_output.json'
assert movie_path.is_file()
assert roi_path.is_file()

In [ ]:
def create_dict_of_rois(roi_list, norm=1, offset=0):
    dex = []
    for roi in roi_list:
        r = np.sqrt(roi.x0**2+roi.y0**2)
        theta = np.arctan2(roi.y0, roi.x0)
        if theta<0.0:
            theta += 2.0*np.pi
        dex.append(r+theta)
    sorted_dex = np.argsort(dex)
    new_roi_dict = {}
    
    ct = 0
    for ct, ii in enumerate(sorted_dex):
        old_roi = roi_list[ii]
        new_roi = OphysROI(roi_id=int(norm*ct)+offset,
                           x0=old_roi.x0,
                           y0=old_roi.y0,
                           width=old_roi.width,
                           height=old_roi.height,
                           mask_matrix=old_roi.mask_matrix,
                           valid_roi=True)
        assert new_roi.roi_id not in new_roi_dict
        assert new_roi.x0==old_roi.x0
        assert new_roi.y0==new_roi.y0
        new_roi_dict[new_roi.roi_id] = new_roi
    return new_roi_dict

In [ ]:
with open(roi_path, 'rb') as in_file:
    raw_roi = json.load(in_file)

denoised_roi_list = []
for i_roi, roi_data in enumerate(raw_roi):
    roi = OphysROI(height=roi_data['height'],
                   width=roi_data['width'],
                   mask_matrix=roi_data['mask_matrix'],
                   x0=roi_data['x'],
                   y0=roi_data['y'],
                   roi_id=i_roi, valid_roi=True)
    denoised_roi_list.append(roi)
denoised_roi_dict = create_dict_of_rois(denoised_roi_list)
del denoised_roi_list
del raw_roi

In [ ]:
traditional_roi_list = []
input_json_path = data_dir / 'OPHYS_EXTRACT_TRACES_QUEUE_1048483611_input.json'
with open(input_json_path, 'rb') as in_file:
    raw_roi = json.load(in_file)
for i_roi, roi_data in enumerate(raw_roi['rois']):
    assert i_roi>=0
    roi = OphysROI(height=roi_data['height'],
                   width=roi_data['width'],
                   mask_matrix=roi_data['mask'],
                   x0=roi_data['x'],
                   y0=roi_data['y'],
                   roi_id=-1-i_roi, valid_roi=True)
    traditional_roi_list.append(roi)
traditional_roi_dict = create_dict_of_rois(traditional_roi_list, norm=-1, offset=-1)
del raw_roi
del traditional_roi_list

In [ ]:
%%time
examiner = ROIExaminer(movie_path)

In [ ]:
roi_sets = []
roi_sets.append(((255,0,0), list(denoised_roi_dict.values())))
roi_sets.append(((0,255,0), list(traditional_roi_dict.values())))

In [ ]:
%%time
examiner.load_rois_to_compare(roi_sets)

In [ ]:
%%time
fig, ax = plt.subplots(1,1, figsize=(20,20))
examiner.plot_rois([0,1], ax, labels=True)

In [ ]:
%%time
fig, ax = plt.subplots(1,1, figsize=(20,20))
examiner.plot_distinct_rois([0,1], ax, labels=True)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(30,30))
examiner.plot_rois([0,1],axes[0],labels=True, alpha=1.0)
examiner.plot_distinct_rois([0,1],axes[1],labels=True, alpha=1.0)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(30,30))
examiner.plot_rois([0,1],axes[0],labels=True, alpha=1.0)
examiner.plot_overlapping_rois([0,1],axes[1],labels=True, alpha=1.0)

In [ ]:
%%time
examiner.plot_thumbnail_and_trace(-23)
examiner.plot_thumbnail_and_trace(45)
examiner.plot_thumbnail_and_trace(55)